In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
import yaml
import json
import time
from datetime import datetime
import multiprocessing as mp
import statsmodels.api as sm
from functools import partial
# import dolphindb as ddb
from itertools import chain, product 
from threadpoolctl import threadpool_limits 
from sklearn.linear_model import LinearRegression
import asyncpg
# from PNL import *
from flib.utils import read_supplementary_data, read_target_return, transform_column_type
from flib import DailyReader
from utils import *
import flib
import random
# pd.set_option('display.max_rows', None)  # 显示所有行
# from PNL import PNL_STOCK
warnings.filterwarnings("ignore")
# from pilot import *
from tqdm import tqdm
from implied_vol_solver import CDSImpliedVolatilitySolver, CDSQuoteType
from utils import read_financial_data
import sys
from io import StringIO
from contextlib import redirect_stdout
import logging
logging.getLogger('implied_vol_solver').setLevel(logging.WARNING)  # 只显示警告和错误
from tqdm import tqdm

In [24]:
# data loading
path_lst = os.listdir('/home/yicheng/credit/data')
df_all = pd.DataFrame()
for path  in tqdm(path_lst):
    a = pd.read_parquet(f'/home/yicheng/credit/data/{path}')
    use_lst = ['ticker','date','tenor','parspread','convspreard','upfront', 'runningcoupon','primarycoupon', 'cdsrealrecovery','cdsassumedrecovery','carriedforward', 'compositedepth5y',]
    b = a.loc[a['tenor']=='5Y']
    temp = b
    # temp['path'] = path
    df_all = pd.concat([df_all,temp])



100%|██████████| 5/5 [00:07<00:00,  1.48s/it]


In [25]:
# show columns
a.columns.tolist()
# 'runningcoupon',

['ticker',
 'date',
 'batch',
 'shortname',
 'redcode',
 'sector',
 'region',
 'country',
 'avrating',
 'impliedrating',
 'tier',
 'currency',
 'docclause',
 'primarycurve',
 'primarypricetype',
 'runningcoupon',
 'primarycoupon',
 'tenor',
 'parspread',
 'convspreard',
 'upfront',
 'cdsrealrecovery',
 'cdsassumedrecovery',
 'carriedforward',
 'compositedepth5y',
 'primarydepthcontributed',
 'primarydepthpassed',
 'primarydepthpassed1wma',
 'primarydepthpassed1mma',
 'primarydepthpassed2mma',
 'compositepricerating',
 'compositecurverating',
 'hasclearsubmissions',
 'hasquotes',
 'hasbankcontributions',
 'clearingsubmissionsdepthcontr',
 'quotesdepthcontr',
 'bankcontributionsdepthcontr',
 'clearingsubmissionsdepthpassed',
 'quotesdepthpassed',
 'bankcontributionsdepthpassed',
 'dailyupdatefrequency',
 'maxdailymovement',
 '_52wkhigh5y',
 '_52wklow5y',
 'contrstddevpassedrecovery',
 'contrlqrangepassed',
 'contrlqrangepassedrecovery',
 'contr10thpctlpassed',
 'contr10pctlpassedrecovery

In [ ]:
# filter the data whose tenor is 5y and coupon limitation is y
df_all = df_all.loc[(df_all['tenor']=='5Y') & (df_all['primarycoupon']=='Y') & (df_all['primarycurve']=='Y') & (df_all['currency']=='USD')]

In [27]:
# df_all.to_csv('res/df_all.csv')

In [28]:
# all data including multiple value in one day, which has been saved in res/df_all.csv
# data has so many columns, we need to select the columns we need. And here I show all columns, haowen could find it by opening csv file to look it in detail.
df_all

,ticker,date,batch,shortname,redcode,sector,region,country,avrating,impliedrating,...,wklycontr,wklycontr1m,wklycontr1y,creditdv01,riskypv01,irdv01,rec01,dp,jtd,dtz
163,XRX,2022-01-03,EOD,Xerox Corp,9HC533,Technology,N.Amer,United States,BB,BB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,XRX,2022-01-03,EOD,Xerox Corp,9HC533,Technology,N.Amer,United States,BB,BB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,XRX,2022-01-03,EOD,Xerox Corp,9HC533,Technology,N.Amer,United States,BB,BB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,XRX,2022-01-03,EOD,Xerox Corp,9HC533,Technology,N.Amer,United States,BB,BB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,XRX,2022-01-03,EOD,Xerox Corp,9HC533,Technology,N.Amer,United States,BB,BB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61463,XRX,2024-12-31,EOD,Xerox Corp,9HC533,Technology,N.Amer,United States,B,CCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61465,XRX,2024-12-31,EOD,Xerox Corp,9HC533,Technology,N.Amer,United States,B,CCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61467,XRX,2024-12-31,EOD,Xerox Corp,9HC533,Technology,N.Amer,United States,B,CCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61469,XRX,2024-12-31,EOD,Xerox Corp,9HC533,Technology,N.Amer,United States,B,CCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# at this one we need to adjust the method to first()
# df_use = df_all.sort_values(by=['date',],inplace=False).groupby('date').first()

# ensures there is at most one row per date
df_all['date'] = pd.to_datetime(df_all['date'])
assert len(df_all['date']) == len(df_all['date'].unique())
df_use = df_all.sort_values(by=['date',], inplace=False)


In [8]:
long_term_debt = read_financial_data('financials_1049_ytd', 'ytd', '2020-01-01', '2025-01-01')
current_asset = read_financial_data('financials_1008_ytd', 'ytd', '2020-01-01', '2025-01-01')
current_asset_short_term_ratio = read_financial_data('financials_43901_ytd', 'ytd', '2020-01-01', '2025-01-01')
shares = read_financial_data('financials_1070_ytd', 'ytd', '2020-01-01', '2025-01-01')
short_term_debt = current_asset / current_asset_short_term_ratio
financial_debt = long_term_debt + short_term_debt
financial_debt_ratio = financial_debt / shares

In [9]:
close = DailyReader('sp_stock_daily_2010').get_data('vwap')
close_ticker = transform_column_type(df = close, column_type='ticker')
financial_debt_ratio = transform_column_type(df = financial_debt_ratio, column_type='ticker')
df_use.index = pd.to_datetime(df_use.index)

# 使用reindex替代loc，自动处理缺失索引
close_ticker_aligned = close_ticker.reindex(df_use.index)
financial_debt_aligned = financial_debt_ratio.reindex(df_use.index)
close_ticker_aligned['XRX'].name = 'stock_price'
financial_debt_aligned['XRX'].name = 'financial_debt_ratio'


In [10]:
# if we use join , we do not use reindex
close_ticker['XRX'].name = 'stock_price'
financial_debt_ratio['XRX'].name = 'financial_debt_ratio'
df_use.join(close_ticker['XRX'],how='left').join(financial_debt_ratio['XRX'],how='left').dropna(subset=['stock_price','financial_debt_ratio'])

,ticker,tenor,parspread,convspreard,upfront,runningcoupon,primarycoupon,cdsrealrecovery,cdsassumedrecovery,carriedforward,compositedepth5y,stock_price,financial_debt_ratio
date,,,,,,,,,,,,,
2021-01-04,XRX,5Y,0.017354,0.017500,0.034821,0.01,Y,0.4,0.4,0.0,4.0,22.52,24.956666
2021-01-05,XRX,5Y,0.017326,0.017475,0.034700,0.01,Y,0.4,0.4,0.0,4.0,22.84,24.956666
2021-01-06,XRX,5Y,0.017109,0.017250,0.033659,0.01,Y,0.4,0.4,0.0,4.0,23.29,24.956666
2021-01-07,XRX,5Y,0.016987,0.017125,0.033055,0.01,Y,0.4,0.4,0.0,5.0,23.56,24.956666
2021-01-08,XRX,5Y,0.017039,0.017175,0.033248,0.01,Y,0.4,0.4,0.0,4.0,23.61,24.956666
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,XRX,5Y,0.059070,0.063053,0.189304,0.01,Y,0.4,0.4,0.0,4.0,9.26,26.615634
2024-12-26,XRX,5Y,0.057494,0.061244,0.183987,0.01,Y,0.4,0.4,0.0,3.0,9.28,26.615634
2024-12-27,XRX,5Y,0.057617,0.061384,0.184377,0.01,Y,0.4,0.4,0.0,5.0,9.21,26.615634


In [11]:
# if we use concat 
df = pd.concat([df_use,close_ticker_aligned['XRX'],financial_debt_aligned['XRX']],axis=1).dropna()

the res is the same between join and concat


In [12]:
# the final concat data
df

,ticker,tenor,parspread,convspreard,upfront,runningcoupon,primarycoupon,cdsrealrecovery,cdsassumedrecovery,carriedforward,compositedepth5y,stock_price,financial_debt_ratio
date,,,,,,,,,,,,,
2021-01-04,XRX,5Y,0.017354,0.017500,0.034821,0.01,Y,0.4,0.4,0.0,4.0,22.52,24.956666
2021-01-05,XRX,5Y,0.017326,0.017475,0.034700,0.01,Y,0.4,0.4,0.0,4.0,22.84,24.956666
2021-01-06,XRX,5Y,0.017109,0.017250,0.033659,0.01,Y,0.4,0.4,0.0,4.0,23.29,24.956666
2021-01-07,XRX,5Y,0.016987,0.017125,0.033055,0.01,Y,0.4,0.4,0.0,5.0,23.56,24.956666
2021-01-08,XRX,5Y,0.017039,0.017175,0.033248,0.01,Y,0.4,0.4,0.0,4.0,23.61,24.956666
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,XRX,5Y,0.059070,0.063053,0.189304,0.01,Y,0.4,0.4,0.0,4.0,9.26,26.615634
2024-12-26,XRX,5Y,0.057494,0.061244,0.183987,0.01,Y,0.4,0.4,0.0,3.0,9.28,26.615634
2024-12-27,XRX,5Y,0.057617,0.061384,0.184377,0.01,Y,0.4,0.4,0.0,5.0,9.21,26.615634


In [13]:
df.to_csv('res/input_data.csv')

In [ ]:
# implied vol calculation
# S = 100.0        # stock price
# D = 150.0        # debt per share
t = 5.0          # CDS tenor
r = 0.05         # risk free rate
R = 0.4          # recovery rate
L = 0.5          # loss given default
lamb = 0.3       # barrier deviation
notional = 100_000_000        
cds_coupon = 0.01

# create new columns
df['implied_vol'] = np.nan 
df['par_spread_error'] = np.nan
solver = CDSImpliedVolatilitySolver(t, r, L, lamb, notional, CDSQuoteType.UPFRONT)

with redirect_stdout(StringIO()):
    for i, row in tqdm(df.iterrows()):
        # imply vol by CDS upfront
        market_cds_upfront = row['upfront']
        cds_coupon = row['runningcoupon']
        R = row['cdsassumedrecovery']
        D = row['financial_debt_ratio']
        S = row['stock_price']
        implied_vol, error = solver.solve_implied_volatility(S, D, market_cds_upfront, cds_coupon, R, method='brent')
        df.at[i, 'implied_vol'] = implied_vol
        df.at[i, 'par_spread_error'] = error




1003it [00:04, 220.23it/s]


In [ ]:
# greek calculation - use the same inputs consistent with the implied vol calculation
# t = 5.0          # CDS期限
# r = 0.05         # 无风险利率
# R = 0.4          # 回收率
# L = 0.5          # 债务回收率
# lamb = 0.3       # 违约壁垒参数
# notional = 100000000        
# cds_coupon = 0.01


solver = CDSImpliedVolatilitySolver(t, r, L, lamb, notional, cds_coupon)
df['delta'] = np.nan
df['gamma'] = np.nan

with redirect_stdout(StringIO()):
    for i, row in tqdm(df.iterrows()):
        # param setting
        S = row['stock_price']
        h = max(1e-4 * S, 1.0)   # 例如：价格的万分之一与 1 取大者

        f_up = solver.calculate_cds_spread_continuous(S + h, row['financial_debt_ratio'], row['implied_vol'], row['cdsassumedrecovery'])

        # f(S) —— 你已有的市场或基准值
        f_mid = row['upfront']

        # f(S-h)
        f_dn = solver.calculate_cds_spread_continuous(S - h, row['financial_debt_ratio'], row['implied_vol'], row['cdsassumedrecovery'])

        # 中心差分 Delta / Gamma
        delta = (f_up - f_dn) / (2.0 * h)
        gamma = (f_up - 2.0 * f_mid + f_dn) / (h ** 2)

        df.at[i, 'delta'] = delta
        df.at[i, 'gamma'] = gamma

1003it [00:00, 1151.35it/s]


In [16]:
df.delta.describe()

count    1003.000000
mean       -0.012154
std         0.004790
min        -0.026728
25%        -0.014043
50%        -0.011690
75%        -0.008606
max        -0.004898
Name: delta, dtype: float64

In [17]:
today = datetime.now().strftime('%Y%m%d')
df.to_csv(f'res/output_data.csv')

## Draft Files


In [18]:
fin_ytd_info = read_supplementary_data('stock', 'financials_ytd_dataitem')
fin_ltm_info = read_supplementary_data('stock', 'financials_ltm_dataitem')
fin_info = pd.concat([fin_ytd_info, fin_ltm_info], axis=0)
fin_info

,Table name,Defination,Dataitemid,Type,Source
0,financials_1_ytd,Cost Of Revenues,1,ytd,Income Statement
1,financials_2_ytd,"Depreciation & Amortization, Total - (IS)",2,ytd,Income Statement
2,financials_4_ytd,"EBT, Excl. Unusual Items",4,ytd,Income Statement
3,financials_6_ytd,Earnings Before Xtra Ord. Item & Accounting Ch...,6,ytd,Income Statement
4,financials_7_ytd,Earnings From Continuing Operations,7,ytd,Income Statement
...,...,...,...,...,...
653,financials_49175_ltm,"Net Debt/EBITDA, Excl. Operating Lease Adjustment",49175,ltm,Calculated Intra-Period Ratio
654,financials_49182_ltm,"Net Debt/(EBITDA, Excl. Operating Lease Adjust...",49182,ltm,Calculated Intra-Period Ratio
655,financials_49183_ltm,"Total Debt/(EBITDA, Excl. Operating Lease Adju...",49183,ltm,Calculated Intra-Period Ratio
656,financials_100689_ltm,EBITA,100689,ltm,Income Statement


In [19]:
fin_info.loc[fin_info['Defination'].str.contains('Shares')]

,Table name,Defination,Dataitemid,Type,Source
44,financials_342_ytd,Diluted Weighted Average Shares Outstanding,342,ytd,Weighted Average Shares
85,financials_1070_ytd,Total Common Shares Outstanding,1070,ytd,Balance Sheet Shares
86,financials_1072_ytd,Total Shares Outstanding on Filing Date,1072,ytd,Filing Cover Shares
97,financials_1100_ytd,Common Shares Outstanding,1100,ytd,Balance Sheet Shares
119,financials_1237_ytd,Shares Outstanding on Filing Date,1237,ytd,Filing Cover Shares
149,financials_2024_ytd,Net Common Shares Issued / Bought Back,2024,ytd,Cash Flow
168,financials_2075_ytd,Common Shares Issued / Bought Back - (CF - Fin...,2075,ytd,Cash Flow
170,financials_2087_ytd,Common Shares for Exercise of Stock Options,2087,ytd,Cash Flow
253,financials_3217_ytd,Basic Weighted Average Shares Outstanding,3217,ytd,Weighted Average Shares
518,financials_24152_ytd,ECS Total Common Shares Outstanding,24152,ytd,Balance Sheet Shares


In [20]:
fin_info.loc[fin_info['Table name'].isin(['financials_1049_ytd','financials_1008_ytd','financials_43901_ytd','financials_1070_ytd'])]

,Table name,Defination,Dataitemid,Type,Source
63,financials_1008_ytd,Total Current Assets,1008,ytd,Balance Sheet
77,financials_1049_ytd,Long-Term Debt,1049,ytd,Balance Sheet
85,financials_1070_ytd,Total Common Shares Outstanding,1070,ytd,Balance Sheet Shares
600,financials_43901_ytd,Current Assets / Short Term Debt,43901,ytd,Calculated Intra-Period Ratio


In [21]:
def read_financial_data(field_name, table_name ,start_date, end_date):
    """
    Read financial data from DolphinDB database.
    
    Args:
        field_name (str): The name of the field to read.
        start_date (str): The start date of the data to read.
        end_date (str): The end date of the data to read.
        
    Returns:
    """ 
    data = DailyReader(rf'sp_financials_{table_name}').get_data(field_name, column_type='tradingitemid').loc[start_date:end_date]
    return data

In [22]:
long_term_debt = read_financial_data('financials_1049_ytd', 'ytd', '2020-01-01', '2025-01-01')
current_asset = read_financial_data('financials_1008_ytd', 'ytd', '2020-01-01', '2025-01-01')
ratio = read_financial_data('financials_43901_ytd', 'ytd', '2020-01-01', '2025-01-01')
shares = read_financial_data('financials_1070_ytd', 'ytd', '2020-01-01', '2025-01-01')
short_term_debt = current_asset / ratio
financial_debt = long_term_debt + short_term_debt
financial_debt_ratio = financial_debt / shares

In [23]:
financials_1049_ytd
financials_1008_ytd	current asset
financials_43901_ytd ratio


SyntaxError: invalid syntax (2822487968.py, line 2)

In [ ]:
a = pd.read_parquet('/home/yicheng/credit/data/mkidwsfpi2qmdzop.parquet')
use_lst = ['ticker','date','tenor','parspread','upfront']
b = a.loc[a['tenor']=='5Y']

In [ ]:
set(use_lst) - set(b.columns)

set()

In [ ]:
b[use_lst]

,ticker,date,tenor,parspread,upfront
162,XRX,2021-01-01,5Y,0.017650,0.036259
163,XRX,2021-01-01,5Y,0.017650,-0.153332
164,XRX,2021-01-01,5Y,0.017812,0.037005
165,XRX,2021-01-01,5Y,0.017812,-0.152473
166,XRX,2021-01-01,5Y,0.017812,0.037005
...,...,...,...,...,...
63241,XRX,2021-12-31,5Y,0.020569,-0.135467
63242,XRX,2021-12-31,5Y,0.020569,0.048645
63243,XRX,2021-12-31,5Y,0.020569,-0.135467
63244,XRX,2021-12-31,5Y,0.020569,0.048645


In [ ]:
reader = DailyReader('option_factors_henry')

In [ ]:
reader.field_idx

['stock_tot_volume',
 'stock_tot_amount',
 'so_vol_ratio',
 'log_so_vol_ratio',
 'option_tot_volume',
 'pcratio',
 'tot_open_interest',
 'iv_slope',
 'skewiv',
 'minute_rv',
 'daily_rv',
 'C_1_A_iv',
 'C_1_B_iv',
 'C_1_C_iv',
 'C_1_D_iv',
 'C_1_E_iv',
 'C_1_F_iv',
 'C_1_G_iv',
 'C_1_H_iv',
 'C_1_I_iv',
 'C_1_J_iv',
 'C_1_K_iv',
 'C_1_L_iv',
 'C_1_M_iv',
 'C_1_N_iv',
 'C_1_O_iv',
 'C_1_P_iv',
 'C_1_Q_iv',
 'C_2_A_iv',
 'C_2_B_iv',
 'C_2_C_iv',
 'C_2_D_iv',
 'C_2_E_iv',
 'C_2_F_iv',
 'C_2_G_iv',
 'C_2_H_iv',
 'C_2_I_iv',
 'C_2_J_iv',
 'C_2_K_iv',
 'C_2_L_iv',
 'C_2_M_iv',
 'C_2_N_iv',
 'C_2_O_iv',
 'C_2_P_iv',
 'C_2_Q_iv',
 'C_3_A_iv',
 'C_3_B_iv',
 'C_3_C_iv',
 'C_3_D_iv',
 'C_3_E_iv',
 'C_3_F_iv',
 'C_3_G_iv',
 'C_3_H_iv',
 'C_3_I_iv',
 'C_3_J_iv',
 'C_3_K_iv',
 'C_3_L_iv',
 'C_3_M_iv',
 'C_3_N_iv',
 'C_3_O_iv',
 'C_3_P_iv',
 'C_3_Q_iv',
 'P_1_A_iv',
 'P_1_B_iv',
 'P_1_C_iv',
 'P_1_D_iv',
 'P_1_E_iv',
 'P_1_F_iv',
 'P_1_G_iv',
 'P_1_H_iv',
 'P_1_I_iv',
 'P_1_J_iv',
 'P_1_K_iv',
 'P_1_L

In [ ]:
pd.read_csv('/home/yicheng/credit/res/XRX.csv',index_col=0)

,ticker,tenor,parspread,convspreard,upfront,runningcoupon,primarycoupon,close_price,financial_debt_ratio,implied_vol,par_spread_error,delta,gamma
date,,,,,,,,,,,,,
2021-01-04,XRX,5Y,0.017354,0.016799,-0.154583,0.05,N,22.46,24.956666,0.256716,6.922901e-11,-0.001545,0.000164
2021-01-05,XRX,5Y,0.017326,0.016759,-0.154758,0.05,N,22.85,24.956666,0.259459,6.675119e-11,-0.001512,0.000157
2021-01-06,XRX,5Y,0.017109,0.017250,0.033659,0.01,Y,23.20,24.956666,0.260941,8.407833e-11,-0.001473,0.000151
2021-01-07,XRX,5Y,0.016987,0.017125,0.033055,0.01,Y,23.58,24.956666,0.263054,9.240652e-11,-0.001438,0.000145
2021-01-08,XRX,5Y,0.017039,0.017175,0.033248,0.01,Y,23.55,24.956666,0.263098,8.646947e-11,-0.001443,0.000146
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,XRX,5Y,0.059070,0.059685,0.034990,0.05,N,9.23,26.615634,0.217273,1.822987e-08,-0.009113,0.001929
2024-12-26,XRX,5Y,0.057494,0.057988,0.029030,0.05,N,9.17,26.615634,0.212361,2.775899e-08,-0.009053,0.001937
2024-12-27,XRX,5Y,0.057617,0.058121,0.029495,0.05,N,9.15,26.615634,0.212232,2.763587e-08,-0.009088,0.001948


In [ ]:
pd.read_csv('/home/yicheng/credit/res/res_20250916.csv',index_col=0)

,ticker,tenor,parspread,convspreard,upfront,runningcoupon,primarycoupon,close_price,financial_debt_ratio,implied_vol,par_spread_error
date,,,,,,,,,,,
2021-01-04,XRX,5Y,0.017354,0.016799,-0.154583,0.05,N,22.46,24.956666,0.256716,6.922901e-11
2021-01-05,XRX,5Y,0.017326,0.016759,-0.154758,0.05,N,22.85,24.956666,0.259459,6.675119e-11
2021-01-06,XRX,5Y,0.017109,0.017250,0.033659,0.01,Y,23.20,24.956666,0.260941,8.407833e-11
2021-01-07,XRX,5Y,0.016987,0.017125,0.033055,0.01,Y,23.58,24.956666,0.263054,9.240652e-11
2021-01-08,XRX,5Y,0.017039,0.017175,0.033248,0.01,Y,23.55,24.956666,0.263098,8.646947e-11
...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,XRX,5Y,0.059070,0.059685,0.034990,0.05,N,9.23,26.615634,0.217273,1.822987e-08
2024-12-26,XRX,5Y,0.057494,0.057988,0.029030,0.05,N,9.17,26.615634,0.212361,2.775899e-08
2024-12-27,XRX,5Y,0.057617,0.058121,0.029495,0.05,N,9.15,26.615634,0.212232,2.763587e-08
